In [1]:
# ============================================================
# SETUP & IMPORT
# ============================================================
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import preprocess_input
import numpy as np
import os
from tqdm import tqdm

# Cek GPU
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# PATH CONFIGURATION
# Pastikan path ini sesuai dengan struktur folder Anda
IMAGE_DIR = r'D:\CODE\Image-Captioning-Project\data\Images'
OUTPUT_DIR = r'D:\CODE\Image-Captioning-Project\data\Images'


Num GPUs Available:  0


In [2]:
# ============================================================
# MODEL DEFINITION
# ============================================================
def load_inception_model():
    base_model = InceptionV3(weights='imagenet', include_top=False)
    
    # Kita ambil input model dan output layer terakhir
    new_input = base_model.input
    hidden_layer = base_model.layers[-1].output 

    model = Model(inputs=new_input, outputs=hidden_layer)
    return model

model = load_inception_model()
print("Model Loaded.")
print(f"Output Shape expected: (None, 8, 8, 2048)")



87910968/87910968 [==============================] - 82s 1us/step
Model Loaded.
Output Shape expected: (None, 8, 8, 2048)


In [3]:
# ============================================================
# PREPROCESSING FUNCTION
# ============================================================
def load_image(image_path):
    # Baca file gambar
    img = tf.io.read_file(image_path)
    # Decode jpeg (3 channels)
    img = tf.image.decode_jpeg(img, channels=3)
    # Resize wajib ke 299x299 untuk InceptionV3
    img = tf.image.resize(img, (299, 299))
    # Preprocess (scaling -1 to 1)
    img = preprocess_input(img)
    return img, image_path

In [4]:
# ============================================================
# EXTRACTION LOOP
# ============================================================

# Ambil semua file jpg di folder
all_images = [os.path.join(IMAGE_DIR, f) for f in os.listdir(IMAGE_DIR) if f.endswith('.jpg')]
# Hapus duplikat dan sort
encode_train = sorted(set(all_images))

print(f"Total images to process: {len(encode_train)}")

# Buat TF Dataset agar loading cepat & efisien memori
image_dataset = tf.data.Dataset.from_tensor_slices(encode_train)
image_dataset = image_dataset.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
# Batching: Proses 16/32 gambar sekaligus
image_dataset = image_dataset.batch(32)

print("Starting Feature Extraction")
print("Features will be saved as .npy files next to original images.")

for batch_imgs, batch_paths in tqdm(image_dataset):
    # Forward pass ke model
    batch_features = model(batch_imgs)
    
    # Reshape dari (Batch, 8, 8, 2048) menjadi (Batch, 64, 2048)
    batch_features = tf.reshape(batch_features, (batch_features.shape[0], -1, batch_features.shape[3]))

    # Simpan per file
    for bf, path in zip(batch_features, batch_paths):
        path_of_feature = path.numpy().decode("utf-8")
        # Nama file output: image_name.jpg.npy
        np.save(path_of_feature + '.npy', bf.numpy())

print("Extraction Completed!")

Total images to process: 8091
Starting Feature Extraction
Features will be saved as .npy files next to original images.


100%|██████████| 253/253 [23:30<00:00,  5.58s/it]

Extraction Completed!
